## FashionMNIST DataSet & DataLoader

In [19]:
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [20]:
train_data = datasets.FashionMNIST(
    root="../../data", train=True, download=True, transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="../../data", train=False, download=True, transform=ToTensor()
)

In [21]:
train_dataloader = DataLoader(train_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=False)

## Modeling

In [22]:
from torch import nn


class NeuralNetwork(nn.Module):
    def __init__(self):
        # nn.Module 생성
        super(NeuralNetwork, self).__init__()

        # 레이어 정의
        self.flatten = nn.Flatten()  # nn 내의 Flatten 클래스 이용

        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 128), nn.ReLU(), nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        y = self.linear_relu_stack(x)

        return y

## 모델 생성

In [23]:
import torch

if torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

# device = "cpu"

print(device)

mps


In [24]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=10, bias=True)
  )
)


## Train Model

In [25]:
loss_fn = nn.CrossEntropyLoss()  # 이미 여기에 소프트맥스 함수가 포함되어 있다.
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [26]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)

    model.train()

    for batch, (X, y) in enumerate(dataloader):
        # DataLoader 에 들어있는 tensor 들을 모델과 같은 위치에 두기
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()  # 이전 배치의 기울기 제거
        loss.backward()
        optimizer.step()

        # 배치가 100번 돌 때마다 화면에 출력
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"Train Loss : {loss:>7f} [ {current:>5d} / {size:>5d} ]")

In [27]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)

    # loss는 배치 별로 계산, correct는 전체 데이터 세트에 대한 평균 정확도
    test_loss, correct = 0, 0

    # 모델을 추론 모드로 바꿔준다.
    model.eval()

    # 추론 과정에서는 기울기를 구할 필요 없음
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)

            test_loss += loss_fn(pred, y).item()

            # 10개의 예측값 중 가장 큰 곳의 인덱스를 argmax 로 찾고, target 과 일치하는지 확인
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    num_batches = len(dataloader)
    test_loss /= num_batches
    correct /= size

    print(
        f"Test Error : \n Accuracy : {(100*correct):>0.1f}%, Avg Loss : {test_loss:>8f}\n"
    )

In [28]:
epochs = 10

for i in range(epochs):
    print(f"Epoch {i+1}\n........................")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)

print("Experiment Successful")

Epoch 1
........................
Train Loss : 2.294576 [     0 / 60000 ]
Train Loss : 1.472597 [  6400 / 60000 ]
Train Loss : 0.927392 [ 12800 / 60000 ]
Train Loss : 0.917044 [ 19200 / 60000 ]
Train Loss : 0.829486 [ 25600 / 60000 ]
Train Loss : 0.672047 [ 32000 / 60000 ]
Train Loss : 0.812730 [ 38400 / 60000 ]
Train Loss : 0.700812 [ 44800 / 60000 ]
Train Loss : 0.572073 [ 51200 / 60000 ]
Train Loss : 0.630962 [ 57600 / 60000 ]
Test Error : 
 Accuracy : 78.7%, Avg Loss : 0.626356

Epoch 2
........................
Train Loss : 0.704034 [     0 / 60000 ]
Train Loss : 0.433490 [  6400 / 60000 ]
Train Loss : 0.640886 [ 12800 / 60000 ]
Train Loss : 0.389475 [ 19200 / 60000 ]
Train Loss : 0.587698 [ 25600 / 60000 ]
Train Loss : 0.571984 [ 32000 / 60000 ]
Train Loss : 0.630846 [ 38400 / 60000 ]
Train Loss : 0.800338 [ 44800 / 60000 ]
Train Loss : 0.466468 [ 51200 / 60000 ]
Train Loss : 0.474491 [ 57600 / 60000 ]
Test Error : 
 Accuracy : 81.8%, Avg Loss : 0.534396

Epoch 3
..................

## 훈련된 모델의 가중치를 저장 / 불러오기

불러올 곳에서 모델의 구조를 알고 있는 경우 가중치만 저장하면 적은 용량으로 저장하고 불러오는 것이 가능하다
* 단점 : 모델 class 가 없으면 사용할 수 없다

In [29]:
# model.state_dict() : 모델 내에 있는 레이어 별 가중치를 들고 있는 딕셔너리
torch.save(model.state_dict(), "../../data/saved_models/model_weights.pth")

In [30]:
model.state_dict()

OrderedDict([('linear_relu_stack.0.weight',
              tensor([[ 0.0250,  0.0063, -0.0424,  ..., -0.0664, -0.0115,  0.0079],
                      [ 0.0514,  0.0003,  0.0490,  ...,  0.0146,  0.0017, -0.0069],
                      [-0.0597,  0.0021, -0.0734,  ...,  0.0013,  0.0351,  0.0053],
                      ...,
                      [ 0.0384,  0.0221, -0.0080,  ..., -0.0012,  0.0494,  0.0481],
                      [ 0.0424,  0.0124,  0.0047,  ...,  0.0114, -0.0058, -0.0607],
                      [ 0.0575, -0.0579,  0.0144,  ..., -0.0789, -0.0832, -0.0619]],
                     device='mps:0')),
             ('linear_relu_stack.0.bias',
              tensor([-0.1647,  0.1012, -0.0167, -0.2256,  0.1592,  0.0787, -0.0628,  0.2006,
                      -0.0430,  0.0539,  0.0242, -0.0885,  0.1619,  0.1670, -0.0418,  0.1252,
                       0.0539,  0.1613,  0.1228,  0.1240, -0.1476,  0.0523,  0.0990, -0.0581,
                       0.1194,  0.2538,  0.0134, -0.0662,  0.

In [31]:
# 저장된 가중치 파일(pth) 불러오기
new_model = NeuralNetwork().to(device)
print(new_model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=10, bias=True)
  )
)


In [32]:
# 훈련되지 않은 모델로 예측
test_loop(test_dataloader, new_model, loss_fn)

Test Error : 
 Accuracy : 10.5%, Avg Loss : 2.320468



In [33]:
# 이전에 훈련된 가중치를 불러와 새로운 모델에 load
new_model.load_state_dict(torch.load("../../data/saved_models/model_weights.pth"))
test_loop(test_dataloader, new_model, loss_fn)

/var/folders/h0/f86w2r851zn7yj1w80yw932h0000gn/T/ipykernel_519/3047005004.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  new_model.load_state_dict(torch.load("../../dat

Test Error : 
 Accuracy : 85.0%, Avg Loss : 0.423008



## 훈련된 모델 자체를 저장 / 불러오기

* 모델의 구조를 몰라도 사용 가능
* 용량은 많이 필요하다

In [34]:
torch.save(model, "../../data/saved_models/model.pth")

In [35]:
new_model = torch.load("../../data/saved_models/model.pth")
test_loop(test_dataloader, new_model, loss_fn)

/var/folders/h0/f86w2r851zn7yj1w80yw932h0000gn/T/ipykernel_519/2093080944.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  new_model = torch.load("../../data/saved_models

Test Error : 
 Accuracy : 85.0%, Avg Loss : 0.423008

